In [39]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pymongo
from dotenv import load_dotenv, dotenv_values

load_dotenv()
config = dotenv_values(".env")

In [40]:
client = pymongo.MongoClient(config.get("MONGO_URI"))
db = client["tastetube"]

videos = db.videos.find()

video_data = [
    {
        "video_id": str(vid["_id"]),
        "text": f"{vid['title']} {vid['description']} {' '.join(vid['hashtags'])}",
    }
    for vid in videos
]

video_data

[{'video_id': '67d2f5960cde55e10a02a3e0',
  'text': 'Morning Yoga Routine for Beginners A 15-minute yoga session to kickstart your day with flexibility and calm. yoga morningroutine fitness beginners'},
 {'video_id': '67d2f5960cde55e10a02a3e1',
  'text': 'Vegan Dinner Recipes Under 30 Minutes Quick and delicious plant-based meals for busy weeknights. vegan cooking recipes quickmeals'},
 {'video_id': '67d2f5960cde55e10a02a3e2',
  'text': 'Top 5 Gaming Headsets of 2025 Reviewing the best headsets for immersive gaming this year. gaming tech headsets reviews'},
 {'video_id': '67d2f5960cde55e10a02a3e3',
  'text': 'DIY Home Decor on a Budget Transform your space with these affordable DIY projects. diy homedecor budgetfriendly crafts'},
 {'video_id': '67d2f5960cde55e10a02a3e4',
  'text': 'Street Photography Tips for Beginners Learn how to capture stunning urban shots with your camera. photography streetphotography beginners tips'},
 {'video_id': '67d2f5960cde55e10a02a3e5',
  'text': 'How to T

In [5]:
# TF-IDF Vectorization
tfidf = TfidfVectorizer(stop_words="english")
tfidf_matrix = tfidf.fit_transform([v["text"] for v in video_data])

cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

cosine_sim

array([[1.        , 0.        , 0.        , 0.        , 0.14019904,
        0.        , 0.08355188, 0.        , 0.        , 0.        ,
        0.28374971, 0.        , 0.        , 0.        , 0.07508241,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 1.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.20510154, 0.        , 0.        , 0.        ,
        0.        , 0.1544437 , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 1.        , 0.        , 0.        ,
        0.        , 0.        , 0.03012396, 0.02560937, 0.02744199,
        0.        , 0.        , 0.13502797, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.02284679, 0.        ],
       [0.        , 0.        , 0.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0

In [ ]:
def recommend_videos(video_id, num_recommendations=5):
    idx = next(
        (i for i, video in enumerate(video_data) if video["video_id"] == video_id), None
    )
    if idx is None:
        return []
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1 : num_recommendations + 1]  # Exclude self
    video_indices = [i[0] for i in sim_scores]
    return [video_data[i] for i in video_indices]

In [36]:
video_id = "67d2f5960cde55e10a02a3e0"
idx = next(
    (i for i, video in enumerate(video_data) if video["video_id"] == video_id), None
)
print(video_data[idx])
recommend_videos(video_id)

{'video_id': '67d2f5960cde55e10a02a3e0', 'text': 'Morning Yoga Routine for Beginners A 15-minute yoga session to kickstart your day with flexibility and calm. yoga morningroutine fitness beginners'}


[{'video_id': '67d2f5960cde55e10a02a3ea',
  'text': 'Advanced Yoga Poses Tutorial Master challenging poses with this expert guide. yoga fitness advanced tutorial'},
 {'video_id': '67d2f5960cde55e10a02a3e4',
  'text': 'Street Photography Tips for Beginners Learn how to capture stunning urban shots with your camera. photography streetphotography beginners tips'},
 {'video_id': '67d2f5960cde55e10a02a3e6',
  'text': 'Minimalist Skincare Routine Simplify your skincare with these 5 must-have products. skincare minimalism beauty routine'},
 {'video_id': '67d2f5960cde55e10a02a3ee',
  'text': 'Beginner Guitar Lessons Learn your first chords with this easy tutorial. guitar music beginners lessons'},
 {'video_id': '67d2f5960cde55e10a02a3e1',
  'text': 'Vegan Dinner Recipes Under 30 Minutes Quick and delicious plant-based meals for busy weeknights. vegan cooking recipes quickmeals'}]